# Tool Calling 

LLMs can return the tools and theor parameters which they think are suitable for the given prompt.

#### Initialize openai client

In [2]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=os.getenv("OPEN_AI_API_KEY")
)

#### Let's build a simple tool for the LLM to call

In [3]:
def get_weather(latitude: float, longitude: float):
    return f"the temperature at {latitude}, {longitude} is 21C"

#### Let's define and describe the tools

In [4]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature of the current coordinates in celsius",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False,
        },
        "strict": True,
    }
}]

In [7]:
system_prompt = "You are a helpful weather assistant"

In [8]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "what's the weather like in paris"},
]

In [9]:
chat_completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

Model decides to call functions w.r.t the nature of the prompt.

In [10]:
chat_completion.model_dump()

{'id': 'chatcmpl-B8VGW3o8zHzgkwCwtImxqSn84yGYm',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_06TqyzK7z5EuAVScDgyFUACF',
      'function': {'arguments': '{"latitude":48.8566,"longitude":2.3522}',
       'name': 'get_weather'},
      'type': 'function'}]}}],
 'created': 1741366508,
 'model': 'gpt-4o-2024-08-06',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_eb9dce56a8',
 'usage': {'completion_tokens': 25,
  'prompt_tokens': 66,
  'total_tokens': 91,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [11]:
def call_function(name, args):
    if name == "get_weather":
        return get_weather(**args)

In [14]:
import json
for tool_call in chat_completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(chat_completion.choices[0].message)

    result = call_function(name, args)

    messages.append(
        {"role":"tool", "tool_call_id":tool_call.id, "content":json.dumps(result)}
    )

Supply result and call model again

In [15]:
chat_completion2 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

In [16]:
chat_completion2.model_dump()

{'id': 'chatcmpl-B8Va3sVkKTWKQLoxTBII14PMmXaKo',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'The current temperature in Paris is 21°C.',
    'refusal': None,
    'role': 'assistant',
    'audio': None,
    'function_call': None,
    'tool_calls': None}}],
 'created': 1741367719,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_06737a9306',
 'usage': {'completion_tokens': 12,
  'prompt_tokens': 118,
  'total_tokens': 130,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [18]:
chat_completion2.choices[0].message.content

'The current temperature in Paris is 21°C.'